###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $h/J = 1$.

In [1]:
N = 6
Dim = 2^N

J = 1. #this is the exchange
B = 1. #this is the transverse field

Hamiltonian = zeros(Float32,Dim,Dim)   #This is your 2D Hamiltonian matrix

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 #spins are +1 and -1
    end
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
    
end

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [2]:
using LinearAlgebra
Diag = eigen(Hamiltonian)     #Diagonalize the Hamiltonian
println(Diag.values)         #This is all of the eigenvalues
#index = findmin(Diag.values)  #The minimum eigenvalue corresponds to index=1
#println(index[1])
GroundState = Diag.vectors[:, 1]  #this gives the groundstate eigenvector
println(GroundState)

Float32[-7.29623, -6.81408, -5.87781, -5.39566, -5.02397, -4.54182, -4.30219, -3.82004, -3.75441, -3.60555, -3.41247, -3.27226, -3.1234, -2.93032, -2.88377, -2.40162, -2.33599, -2.02993, -1.99404, -1.85384, -1.54778, -1.51189, -1.48215, -1.1402, -0.999999, -0.760362, -0.658056, -0.611506, -0.418418, -0.278216, -0.12936, -0.0637288, 0.0637283, 0.129363, 0.278218, 0.418421, 0.611509, 0.658059, 0.760364, 1.0, 1.1402, 1.48215, 1.5119, 1.54778, 1.85384, 1.99404, 2.02993, 2.33599, 2.40162, 2.88377, 2.93032, 3.12341, 3.27226, 3.41246, 3.60555, 3.75441, 3.82004, 4.30219, 4.54182, 5.02397, 5.39566, 5.87781, 6.81408, 7.29623]
Float32[0.400546, 0.206267, 0.130935, 0.166201, 0.122672, 0.0748228, 0.0952468, 0.156543, 0.122672, 0.0670863, 0.0472049, 0.0676195, 0.0874641, 0.061567, 0.0911077, 0.166202, 0.130935, 0.0695418, 0.0456557, 0.0615668, 0.0472049, 0.0311591, 0.0425391, 0.0748231, 0.0952468, 0.055899, 0.0425391, 0.0670866, 0.0911077, 0.0695421, 0.107974, 0.206269, 0.206267, 0.107974, 0.0695418

In [3]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = 0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    SumSz = 0.
    for SpinIndex = 0:N-1  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        SumSz += Spin1 #spin is +1 or -1
        #print(Spin1," ")
    end
    #println(SumSz," ",GroundState[Ket+1])
    magnetization += SumSz*SumSz*GroundState[Ket+1]^2  #Don't forgot to square the coefficients...
end
println(magnetization/(N*N))    

0.48243552875808543
